In [1]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import matplotlib
from sklearn.ensemble import RandomForestRegressor
import datetime as dt
import csv

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [6]:
def getData():
  data = []
  dateNums = []
  cases = []
  fatalities = []

  first = 0
  firstDate = ''
  countryCurr = ''
  stateCurr = ''

  for i in range(train_data.shape[0]):
    useThis = train_data.iloc[i]["Province_State"]
    if not isinstance(train_data.iloc[i]["Province_State"], str):
      useThis = 'NONE'
    if train_data.iloc[i]["Country_Region"] == countryCurr and useThis == stateCurr:
      cases.append(train_data.iloc[i]["ConfirmedCases"]+cases[i-first])
      fatalities.append(train_data.iloc[i]["Fatalities"]+fatalities[i-first])
      date = dt.datetime.strptime(train_data.iloc[i]["Date"],'%Y-%m-%d')
      delta = date - firstDate
      dateNums.append(delta.days)
    else:
      if countryCurr != '':
        x=dateNums
        # x = np.empty((0,1),int)
        # for i in dateNums:
        #   x = np.append(arr,np.array([[i]]), axis=0)
        data.append([countryCurr, stateCurr, x, cases, fatalities, firstDate])
        dateNums = []
        cases = []
        fatalities = []

      countryCurr = train_data.iloc[i]["Country_Region"]
      stateCurr = train_data.iloc[i]["Province_State"]
      if not isinstance(stateCurr, str):
        stateCurr = 'NONE'

      cases.append(train_data.iloc[i]["ConfirmedCases"])
      fatalities.append(train_data.iloc[i]["Fatalities"])
      date = dt.datetime.strptime(train_data.iloc[i]["Date"],'%Y-%m-%d')
      firstDate = date
      dateNums.append(0)
      first = i+1
  x=dateNums
  # x = np.empty((0,1),int)
  # for i in dateNums:
  #   x = np.append(arr,np.array([[i]]), axis=0)
  data.append([countryCurr, stateCurr, x, cases, fatalities, firstDate])
  return data

data = getData()

In [7]:
# i[0] = country, i[1] = state, i[2] = dates, i[3] = cases, i[4]=fatalities, i[5]=firstDate
def randomForest(data, country, state, date):
  for i in data:
    useThis = state
    if not isinstance(state, str):
      useThis = 'NONE'
    if i[0] == country and i[1] == useThis:
      x = np.empty((0,1),int)
      for j in i[2]:
        x = np.append(x,np.array([[j]]), axis=0)
      date = dt.datetime.strptime(date,'%Y-%m-%d')
      delta = date - i[5]

      regressorC = RandomForestRegressor(n_estimators = 100, random_state = 0) 
      regressorC.fit(x,i[3])
      predC = regressorC.predict(np.array([delta.days]).reshape(1, 1))

      regressorF = RandomForestRegressor(n_estimators = 100, random_state = 0) 
      regressorF.fit(x,i[4])
      predF = regressorF.predict(np.array([delta.days]).reshape(1, 1))
      return float(predC), float(predF)
  return "Not Found", "Not Found"

In [10]:
data = getData()

submitData = []

for i in range(test_data.shape[0]):
  forecastId = test_data.iloc[i]['ForecastId']
  state = test_data.iloc[i]['Province_State']
  country = test_data.iloc[i]['Country_Region']
  date = test_data.iloc[i]['Date']
  predC, predF = randomForest(data, country, state, date)
  print(forecastId, predC, predF)
  if predC == 'Not Found':
    print(country, state, date)
  li = (forecastId, predC, predF)
  submitData.append(li)

Streaming output truncated to the last 5000 lines.
8460 172.92 0.0
8461 181.88 0.0
8462 188.52 0.0
8463 197.0 0.0
8464 204.36 0.0
8465 212.44 0.0
8466 221.88 0.0
8467 230.04 0.0
8468 237.88 0.0
8469 246.12 0.0
8470 253.08 0.0
8471 262.68 0.0
8472 679.92 31.46
8473 764.91 34.34
8474 853.93 37.23
8475 975.81 41.12
8476 1073.88 45.05
8477 1181.68 49.67
8478 1310.26 55.11
8479 1417.4 59.55
8480 1558.9 65.84
8481 1688.11 71.69
8482 1817.67 77.45
8483 1962.47 83.56
8484 2152.48 91.92
8485 2291.28 98.58
8486 2462.12 106.48
8487 2650.06 114.32
8488 2869.34 123.04
8489 3033.4 129.44
8490 3232.44 137.12
8491 3503.99 147.75
8492 3707.35 156.12
8493 3899.65 164.2
8494 4120.25 173.2
8495 4333.12 181.66
8496 4576.95 191.29
8497 4787.48 199.57
8498 5027.22 208.69
8499 5275.86 218.0
8500 5616.41 230.21
8501 5875.36 238.3
8502 6263.13 249.0
8503 6591.2 257.4
8504 7050.54 268.6
8505 7404.92 276.9
8506 7869.15 287.5
8507 8297.38 296.7
8508 8845.59 306.8
8509 9609.32 318.61
8510 10321.71 328.84
8511 11029

In [11]:
with open("submission.csv", "wt") as fp:
    writer = csv.writer(fp, delimiter=",")
    writer.writerow(["ForecastID", "ConfirmedCases", "Fatalities"])  # write header
    writer.writerows(submitData)